# Проект №5 "Выбираем авто" - работа Елены Ондар

## 1. Импорт библиотек

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re
from datetime import datetime

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor
from sklearn.cluster import KMeans
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import cross_validate

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 15
DIR_TRAIN  = '../input/parsingautoru24122020/' # подключим к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

# EDA для тестовой выборки
Так как в этом проекте у нас нет выборки для обучения, а есть только тестовая выборка, необходимо: 
* Изучить данные, представленные в тесте, чтобы понять какие данные надо собирать из внешних источников
* Сформировать желаемый вид датасета, к которому будем стремиться преобразовать собранные данные

### Посмотрим какие данные в тестовой выборке

In [ ]:
test = pd.read_csv(DIR_TEST+'test.csv')
test.head()

In [ ]:
# Определим кол-во строк и столбцов тестового датасета проекта

print('Тестовый датасет проекта содержит {} строк, {} столбцов\n'.format(test.shape[0], test.shape[1]))

In [ ]:
# Подсчитаем количество столбцов по типам данных
print(test.dtypes.value_counts())

In [ ]:
# Определим кол-во столбцов, содержащих пустые значения
print(f'В {test.isnull().any().sum()} столбцах есть отсутствующие значения.\n')

# Посмотрим абс и относит величину пропущенных значений по каждому признаку
cols_isnan=pd.DataFrame({'count': test.isnull().sum(),
                              'ratio': test.isnull().sum()/len(test)}).query('count > 0')
print('Признаки, в которых есть данные с пропущенными значениями\n\n{}'
      .format(cols_isnan))

In [ ]:
def param_data(data): # посмотрим на данные
  param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              'loc[1]': data.loc[1].values,
              }, 
             index = data.loc[0].index)
  return param

In [ ]:
#pd.concat([param_data(asd_99), param_data(a)], axis=1, sort=False)
param_data(test)

In [ ]:
test.columns

In [ ]:
# Переименуем столбцы. 
# Для столбцов, названия которых на английском, все приведем к единому стилю - строчные англ. буквы, 
# разделитель между словами - знак подчеркивания.
# Несколько столбцов имеют названия на русском. Можно сделаь транслитерацию, но так как их немного, то просто переименуем.
def upper_letter(word):
    for i in range(len(word)):
        if word[i].isupper() and i!=0:
            word = word[:i] + ' ' + word[i:].lower()
    return word.replace(' ', '_').lower()

test = pd.read_csv(DIR_TEST+'test.csv')

test.rename(columns=lambda x: upper_letter(x) if re.search('[a-z]', x) else x, inplace=True)

russian_cols = {'Владельцы': 'owners', 'Владение': 'own', 'ПТС': 'pts', 'Привод': 'drive', 
                'Руль': 'rudder', 'Состояние': 'condition', 'Таможня': 'customs'}

test.rename(columns=russian_cols, inplace=True)

test.columns

In [ ]:
# Распределим признаки по категориям: бинарные, категориальные и числовые
test_cols = test.columns#[1:-2]
test_binary_cols, test_category_cols, test_numeric_cols = [], [], []

for j in test_cols:
    if len(test[j].value_counts(dropna=False))==2:
        test_binary_cols.append(j)
    #elif len(test[j].value_counts(dropna=False))<11 or test[j].dtypes==object:
    elif test[j].dtypes==object:
        test_category_cols.append(j)
    else:
        test_numeric_cols.append(j)
            
print('binary_columns =', test_binary_cols, \
      '\ncategory_columns =', test_category_cols, '-',len(test_category_cols), 
      '\nnumeric_columns =', test_numeric_cols)

### Посмотрим на данные, которые находятся в столбцах vendor и rudder

In [ ]:
for column in test_binary_cols:
    display(test[column].value_counts())

### Посмотрим на данные в категориальных признаках

In [ ]:
for column in test_category_cols:
    display(test[column].value_counts(dropna=False))

### Посмотрим, что можно сделать с body_type

In [ ]:
test['body_type'].value_counts(dropna=False)

Признаки 'body_type', 'brand' и 'color' пока трогать не буду.
Признак body_type пока не трогаю. Посмотрю потом, что будет в спарсенных данных.
В 'brand' всего 12 основых автомобильных брэндов, это не так много.
В 'color' - 16 различных цветов. Можно оставить 10 основных цветов, но это можно сделать потом, посмотрев на данные, которые мы возьмем извне.

### Посмотрим, какие данные содержатся в 'complectation_dict'

In [ ]:
test['complectation_dict'].value_counts(dropna=False)

Очень много пустых значений - 28268 (81%). Скорее всего информация о полной комплектации автомобиля. Может и можно было бы вытащить какие-нибудь признаки, но пока это делать не будем, так как слишком много пропущенных данных.

### Посмотрим признак 'description'

In [ ]:
test['description'].value_counts(dropna=False)

Из признака description (описание) можем получить информацию, продажа идет через дилера или от самого собственника, и о возможности оформления в кредит.
### Дальше идут признаки 'engine_displacement' - объем двигателя, 'engine_power' - мощность двигателя

In [ ]:
test['engine_displacement'].value_counts(dropna=False)

Не знаю, что это за аббревиатура LTR и будет ли встречаться еще какая-либо. Поэтому попробую в отдельном датасете разбить на два отдельных поля и посмотрю, что это.

In [ ]:
engine_displacement_tmp = test['engine_displacement'].str.split(" ", expand = True)
param_data(engine_displacement_tmp)

Есть одна запись, где объема двигателя нет, только аббревиатрура. Этой записи присвоим значение 0.
Затем оставим только значение объема двигателя, без аббревиатуры (она одинаковая для всех записей), и приведем к числовому типу.

In [ ]:
test['engine_displacement'] = engine_displacement_tmp[0]
test.loc[test['engine_displacement'] == '', 'engine_displacement'] = 0
test['engine_displacement'] = test['engine_displacement'].astype(float)

In [ ]:
test['engine_power'].value_counts(dropna=False)

В признаке 'engine_power' также есть аббревиатура. Посмотрим, какие варианты аббревиатуры встречаются.

In [ ]:
engine_power_tmp = test['engine_power'].str.split(" ", expand = True)
param_data(engine_power_tmp)

Поступим аналогично с engine_displacement - оставим только значение мощности двигателя, без аббревиатуры (она одинаковая для всех записей), и приведем к числовому типу.

In [ ]:
test['engine_power'] = engine_power_tmp[0]
test['engine_power'] = test['engine_power'].astype(int)

### Посмотрим признак 'equipment_dict'

In [ ]:
test['equipment_dict'].value_counts(dropna=False)

Идут какие-то характеристики: круиз-контроль, тонированные стекла, usb, т.е. комплектация автомобиля.
Пропусков 9996 (29%), меньше чем в признаке complectation_dict.
Нашла в одной из работ, ребята вытаскивали данные из этого признака. Воспользуюсь их наработкой.

In [ ]:
# Функция, которая сделает полный список всех возможных характеристик в тестовой выборке.

def get_test_features(equipment):
    # Создаем пустой список, в который будут добавляться все характеристики
    all_features = []
    for data in equipment:
        # Находим все слова между кавычками
        features=re.findall(r'\"(.+?)\"',data)
        # Добавляем в общий список
        all_features.extend(features)
    # Удаляем дубликаты
    all_features = list(dict.fromkeys(all_features))
    return all_features

In [ ]:
test_features = get_test_features(test[test.equipment_dict.isna()==False].equipment_dict)
len(test_features)

In [ ]:
test[test.equipment_dict.isna()==False].head()

In [ ]:
# # Удаляем лишние записи
# for bad_feature in ['name','Безопасность','values','Комфорт','Мультимедиа','Обзор','Салон','Защита от угона','Элементы экстерьера']:
#     test_features.remove(bad_feature)  

print('Всего уникальных признаков:', len(test_features))
print(test_features)

In [ ]:
# Заменим значение параметра equipment в тестовой выборке на список характеристик.
def get_features_test(equipment): 
    features=re.findall(r'\"(.+?)\"',equipment)  
    return features

test.loc[test.equipment_dict.isna()==False, 'equipment_dict'] = test[test.equipment_dict.isna()==False]['equipment_dict'].apply(lambda x: get_features_test(x))
test[test.equipment_dict.isna()==False].sample(5)

### Рассмотрим признак 'fuel_type'

In [ ]:
test['fuel_type'].value_counts(dropna=False)

Этот признак содержит тип используемого топлива, оставим как есть.

### Рассмотрим признак model_info

In [ ]:
test['model_info'].value_counts(dropna=False)

In [ ]:
# Посмотрим признак model_name
dict_model = dict(test['model_name'].value_counts())
dict_model
len(dict_model)
#model_name = test.loc[mask>1]['model_name'].value_counts()
#model_name

In [ ]:
# Посмотрим признак name
test['name'].value_counts(dropna=False)

Думаю, что в конце концов надо удалить признаки model_info, model_name, name, так как они не дают новой информации

### Рассмотрим признак price_currency

In [ ]:
test['price_currency'].value_counts(dropna=False)

In [ ]:
Во всех записях указана одна валюта - рубли. Признак можно будет удалить.

### Следующий признак super_gen

In [ ]:
test['super_gen'].value_counts(dropna=False)

Этот признак содержит большую часть признаков, которые были по другим реквизитам. Воспользуемся им для заполнения некоторых пустых значений, может быть.

### Следующий рассматриваемый признак vehicle_configuration

In [ ]:
test['vehicle_configuration'].value_counts()

### Посмотрим, какие данные содержатся в vehicle_transmission

In [ ]:
test['vehicle_transmission'].value_counts()

vehicle_transmission - трансмиссия (коробка передач)


### Посмотрим, какие данные содержатся в owners и own

In [ ]:
test['owners'].value_counts()

Признак owner показывает нам сколько раз машина переходила из рук в руки. Обычно, если машина поменяла слишком много владельцев, то такая машина стоит дешевле, чем та у которой был всего один владелец.

In [ ]:
# Посмотрим значения в виде списка
test.owners.unique()

In [ ]:
# Обработка owners: сделаю словарь, 
dict_owners = {'3 или более': 3, '1\xa0владелец': 1, '2\xa0владельца': 2}

# заменяю значение признака owners, соответствующее ключу dict_owners значением данного ключа
test['owners'] = test['owners'].map(dict_owners)
test.owners.value_counts()

# преобразую тип данных к int
test['owners'] = test['owners'].astype(int)

### Рассмотрим признак own:

In [ ]:
test['own'].value_counts()

Думаю, что признак own нам не особо нужен, так как у нас есть информация о годе выпуска автомобиля и количестве владельцев. К тому же слишком много пропусков - 22691, т.е. не заполнено 65% данных.

In [ ]:
# top_body_type = test['body_type'].value_counts()[:10]
# test['body_type'] = test.body_type.apply(lambda x: x if x in top_body_type.keys() else 'Другой')
# test['body_type'].value_counts(dropna=False)

own = test['own'].value_counts().reset_index()
own[own['own']<=1000].count()

### Рассмотрим признак pts

In [ ]:
test['pts'].value_counts(dropna=False)

In [ ]:
# Посмотрим, какая информация есть по записи с пропуском в pts
test[test['pts'].isna()]

In [ ]:
dict(test[test['pts'].isna()]['super_gen'])

In [ ]:
test[test['pts'].isna()]['description'].tolist()

In [ ]:
# Заполним пропущенное значение самым частовстречающимся
test.loc[test['pts'].isna(), 'pts'] = test['pts'].mode()[0]


In [ ]:
test[test.parsing_unixtime==1603118960]

### Рассмотрим признак drive

In [ ]:
test['drive'].value_counts()

### Рассмотрим два последних категориальных признака: condition и customs

In [ ]:
test['condition'].value_counts()

In [ ]:
test['customs'].value_counts()

Признаки condition и customs содержат всего по одному значению. Посмотрим в трейне, если там не будет каких-либо других значений, то признаки можно будет удалить.

### Посмотрим на данные, которые содержатся в числовых признаках

In [ ]:
for column in test_numeric_cols:
    display(test[column].value_counts())

Признак number_ofdoors - скорее категориальный признак, parsing_unixtime - попробуем преобразовать в дату

In [ ]:
# Преобразуем parsing_unixtime в дату
test['parsing_date']=pd.to_datetime(test['parsing_unixtime'], unit='s')
test['parsing_date']

In [ ]:
# Перераспределим признаки по категориям: категориальные и числовые
test_cols = test.columns#[1:-2]
test_category_cols, test_numeric_cols = [], []

for j in test_cols:
    if len(test[j].value_counts(dropna=False))<11 or test[j].dtypes==object:
        test_category_cols.append(j)
    else:
        test_numeric_cols.append(j)
            
print('category_columns =', test_category_cols, '-',len(test_category_cols), 
      '\nnumeric_columns =', test_numeric_cols)

In [ ]:
# Посмотрим на распределение числовых данных:
fig, axes = plt.subplots(nrows=2, ncols=(round(len(test_numeric_cols)/2)+1), figsize=(12, 8))
i = 0

for j in test_numeric_cols:
    plot = sns.distplot(test[j], kde=False, ax=axes.flatten()[i])
    plot.set_title(j)
    plot.set_xticklabels([])
    plt.tight_layout()
    i = i + 1

Прологарифмируем и построим графики распределения логарифмированных переменных.

In [ ]:
# Для начала создам функцию по созданию нового признака:


def creating_new_columns(df, column, function):
    if function == 'log':
        df[df[column].name+'_log'] = np.log(df[column][df[column] > 0])
        df[df[column].name+'_log'].fillna(0, inplace=True)
    elif function=='sqrt':
        df[df[column].name+'_sqrt'] = np.sqrt(df[column][df[column] > 0])
        df[df[column].name+'_sqrt'].fillna(0, inplace=True)
    elif function =='isNAN':
        if df[column].isnull().any() == True:
            df[df[column].name+'_isNAN'] = pd.isna(df[column]).astype('uint8')

In [ ]:
# Для каждого числового признака создам новый признак, содержащий логарифмированную величину этой переменной

test_numeric_log, func = [], 'log'

for i in test_numeric_cols:
    creating_new_columns(test, i, func)
    test_numeric_log.append(i+'_'+func)
    
test[test_numeric_cols+test_numeric_log].head(10)

In [ ]:
# Посмотрим распределение логарифмов
fig, axes = plt.subplots(nrows=2, ncols=(round(len(test_numeric_log)/2)+1), figsize=(12, 8))
i = 0

for j in test_numeric_log:
    plot = sns.distplot(test[j], kde=False, ax=axes.flatten()[i])
    plot.set_title(j)
    plot.set_xticklabels([])
    plt.tight_layout()
    i = i + 1

In [ ]:
# Построим боксплоты для числовых переменных
fig, axes = plt.subplots(nrows=1, ncols=len(test_numeric_cols), figsize=(17, 6))
i = 0

for j in test_numeric_cols:
    sns.boxplot(y=test[j], showmeans=True, ax=axes.flatten()[i]);#x=test["default"],
    plt.tight_layout()#plt.show()
    i = i+1

In [ ]:
# Выясняем, как ведется нумерация продавцов в зависимости от дня обращения
sns.scatterplot(x='sell_id',y='parsing_unixtime',data=test)

In [ ]:
# Построим боксплоты для логарифмированных переменных
fig, axes = plt.subplots(nrows=1, ncols=len(test_numeric_log), figsize=(17, 6))
i = 0

for j in test_numeric_log:
    sns.boxplot( y=test[j], showmeans=True, ax=axes.flatten()[i]);#x=df["default"], 
    plt.tight_layout()#plt.show()
    i = i+1

In [ ]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru__24_12_2020.csv', sep=';') # датасет для обучения модели
#test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
# Определим кол-во строк и столбцов обучающего датасета проекта
print('Обучающий датасет проекта содержит {} строк, {} столбцов\n'.format(train.shape[0], train.shape[1]))

# Подсчитаем количество столбцов по типам данных
print(train.dtypes.value_counts())

# Определим кол-во столбцов, содержащих пустые значения
print(f'В {train.isnull().any().sum()} столбцах есть отсутствующие значения.\n')

# Посмотрим абс и относит величину пропущенных значений по каждому признаку
cols_isnan=pd.DataFrame({'count': train.isnull().sum(),
                              'ratio': train.isnull().sum()/len(train)}).query('count > 0')
print('Признаки, в которых есть данные с пропущенными значениями\n\n{}'
      .format(cols_isnan))

#pd.concat([param_data(train), param_data(test)], axis=1, sort=False)


In [ ]:
# Удалю строки с пропусками в признаке price обучающей выборки
train.dropna(subset=['price'], inplace=True)

In [ ]:
# Определим кол-во столбцов, содержащих пустые значения
print(f'В {train.isnull().any().sum()} столбцах есть отсутствующие значения.\n')

# Посмотрим абс и относит величину пропущенных значений по каждому признаку
cols_isnan=pd.DataFrame({'count': train.isnull().sum(),
                              'ratio': train.isnull().sum()/len(train)}).query('count > 0')
print('Признаки, в которых есть данные с пропущенными значениями\n\n{}'
      .format(cols_isnan))


In [ ]:
param_data(train)

## Data Preprocessing

In [ ]:
# Распределим признаки обучающего датасета по категориям: бинарные, категориальные и числовые
train_cols = train.columns#[:-2]
train_binary_cols, train_category_cols, train_numeric_cols = [], [], []

for j in train_cols:
    if len(train[j].value_counts(dropna=False))==2:
        train_binary_cols.append(j)
    elif len(train[j].value_counts(dropna=False))<11 or train[j].dtypes==object:
    #elif test[j].dtypes==object:
        train_category_cols.append(j)
    else:
        train_numeric_cols.append(j)
            
print('binary_columns =', train_binary_cols, '\ncategory_columns =', train_category_cols, '-',len(train_category_cols), 
      '\nnumeric_columns =', train_numeric_cols)

In [ ]:
# Сравним данные в train и test
for column in binary_cols:
    display(test[column].value_counts())
    
for column in train_binary_cols:
    display(train[column].value_counts())

In [ ]:
print(train.vendor.value_counts())

In [ ]:
print(train.vehicle_transmission.value_counts())
print(test.vehicle_transmission.value_counts())

In [ ]:
print(train.drive.value_counts())
print(test.drive.value_counts())

In [ ]:
# Заменим русские значения на английские в признаке 
# rudder, pts, fuel_type, vehicle_transmission и drive тестовой выборки
dict_rudder = {'Левый': 'LEFT',
               'Правый': 'RIGHT'}

dict_pts = {'Оригинал': 'ORIGINAL', 
            'Дубликат': 'DUPLICATE'}

dict_test_fuel = {'бензин': 'GASOLINE', 
                  'гибрид': 'HYBRID', 
                  'дизель': 'DIESEL',
                  'электро': 'ELECTRO',
                  'газ': 'LPG'}

dict_transmission = {'автоматическая': 'AUTOMATIC',
                     'механическая': 'MECHANICAL',
                     'роботизированная': 'ROBOT',
                     'вариатор': 'VARIATOR'}

dict_drive = {'передний': 'FORWARD_CONTROL',
              'задний': 'REAR_DRIVE',
              'полный': 'ALL_WHEEL_DRIVE'}


test['rudder'] = test['rudder'].map(dict_rudder)
test['pts'] = test['pts'].map(dict_pts)
test['fuel_type'] = test['fuel_type'].map(dict_test_fuel)
test['vehicle_transmission'] = test['vehicle_transmission'].map(dict_transmission)
test['drive'] = test['drive'].map(dict_drive)

In [ ]:
display(train['body_type'].value_counts())
display(test['body_type'].value_counts())

In [ ]:
# Переведем все символы в нижний регистр и на всякий случай, если есть удалим пробелы в конце и начале.
train['body_type'] = train['body_type'].apply(lambda x: x.strip().lower())
display(train['body_type'].value_counts())
print(train.body_type.nunique())

In [ ]:
# Позаимствовала из работы одного из студентов обработку body_type (Группа DST-17 Владимир Юшманов)

train['bodytype'] = [str(x).lower().replace('.', '') for x in train['body_type']]
test['bodytype'] = [str(x).lower() for x in test['body_type']]

# Заменим длинные типы в train'е на обобщенные значения
body_type_list = list(train['bodytype'].unique())
def get_perf_type(x, body_type_list):
    for t in body_type_list:
        if t in x:
            return t
        else: continue
    else: return '0'
    
train['bodytype'] = train['bodytype'].apply(lambda x: get_perf_type(x, body_type_list))
test['bodytype'] = test['bodytype'].apply(lambda x: get_perf_type(x, body_type_list))

# Удалим 7 предложений, для которых не нашлось соответствия типа кузова
train = train[train['bodytype']!='0']
train['body_type'] = train['bodytype']
test['body_type'] = test['bodytype']

train.drop(columns=['bodytype'], inplace=True)
test.drop(columns=['bodytype'], inplace=True)

In [ ]:
print(test.body_type.unique())
print(train.body_type.unique())

In [ ]:
# Из той же работы обработка color (Группа DST-17 Владимир Юшманов):
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
train['color'] = train['color'].map(dict_color)

In [ ]:
train['color'].value_counts(dropna=False)

При парсинге обратила внимание, что есть признак summary, содержащий данные, которые могут пригодиться.
Для начала посмотрю, какие данные есть в этом признаке

In [ ]:
train['summary']

В summary есть информация об объеме и мощности двигателя, типе кузова, типе привода и используемом топливе.
Данные о типе кузова, типе привода и используемом топливе уже преобразовала в train к виду, соответствующему тестовой выборке. 
Посмотрю, что содержится в engine_displacement, engine_power обучающей выборки.

In [ ]:
train[['engine_displacement', 'engine_power']]

В engine_displacement обучающей выборки данные представлены по-другому, чем в тестовой, поэтому в обучающей выборке engine_displacement заменю данными из summary.

In [ ]:
#  В отдельном датасете разделю данные summary на отдельные поля
summary_tmp = train['summary'].str.replace(',', '').str.split(" ", expand = True)
summary_tmp

In [ ]:
# Определим кол-во столбцов, содержащих пустые значения
print(f'В {summary_tmp.isnull().any().sum()} столбцах есть отсутствующие значения.\n')

# Посмотрим абс и относит величину пропущенных значений по каждому признаку
cols_isnan=pd.DataFrame({'count': summary_tmp.isnull().sum(),
                              'ratio': summary_tmp.isnull().sum()/len(summary_tmp)}).query('count > 0')
print('Признаки, в которых есть данные с пропущенными значениями\n\n{}'
      .format(cols_isnan))

In [ ]:
# Удалю столбцы с 5 по 8
summary_tmp.drop([5,6,7,8], axis=1, inplace=True)

# Очищу столбец с данными объема двигателя от лишних символов
summary_tmp[0] = summary_tmp[0].str.replace('[A-Z,a-z]', '').str.strip()
summary_tmp

In [ ]:
# Посмотрю список уникальных значений
summary_tmp[0].unique()

In [ ]:
summary_tmp.loc[summary_tmp[0] == '']

Есть пустые значения. 
Оставлю в summary_tmp только столбец с объемом двигателя и присоединю к train. 
Заменю данные объема двигателя в обучающей выборке. 
Удалю строки, которые содержат пустые значения объема двигателя.
Удалю лишний столбец.

In [ ]:
summary_tmp = summary_tmp[0]
train = pd.concat([train,summary_tmp], axis=1)

train['engine_displacement'] = train[0]
train = train.loc[train['engine_displacement'] != '']
train['engine_displacement'] = train['engine_displacement'].astype(float)
train.drop([0], axis=1, inplace=True)
train['engine_displacement']

In [ ]:
# Посмотрим на данные в признаке owners
train.owners.value_counts(dropna=False)

In [ ]:
# Обработка owners: 

# Заполню пропущенное значение в признаке pts обучающей выборки значением 0
train['owners'].fillna(0, inplace=True)

# Сделаю словарь
dict_train_owners = {'4.0': 4, '3.0': 3, '2.0': 2, '1.0': 1, '0.0': 0}

# заменяю значение признака owners, соответствующее ключу dict_owners значением данного ключа
train['owners'] = train['owners'].astype('string').map(dict_train_owners)

# преобразую тип данных к int
train['owners'] = train['owners'].astype(int)#apply(lambda x: round(x))#.
print(train.owners.value_counts())
print(train.owners.unique())

In [ ]:
train.number_ofdoors.value_counts(dropna=False)

In [ ]:
# Заполним пропущенное значение в признаке pts обучающей выборки самым частовстречающимся значением
train.loc[train['pts'].isna(), 'pts'] = train['pts'].mode()[0]

In [ ]:
# Перераспределим признаки обучающего датасета по категориям: категориальные и числовые
train_cols = train.columns#[:-2]
train_category_cols, train_numeric_cols = [], []

for j in train_cols:
    if train[j].dtypes!=object:
        train_numeric_cols.append(j)
    else:
        train_category_cols.append(j)
            
print('category_columns =', train_category_cols, '-',len(train_category_cols), 
      '\nnumeric_columns =', train_numeric_cols)

## Посмотрим корреляцию признаков

In [ ]:
train_numeric_cols = ['engine_displacement', 'engine_power', 'mileage', 'model_date', 
                      'number_ofdoors', 'production_date', 'owners']

In [ ]:
corr_matrix = pd.concat([train[train_numeric_cols], train.price], axis=1).corr()
cmap = sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    return [dict(selector="th",
                 props=[("font-size", "10pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "10pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '10pt')])
]

corr_matrix.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
    .set_table_styles(magnify())

In [ ]:
print('Ранг матрицы - {}, det(corr_mat) = {}'.format(np.linalg.matrix_rank(corr_matrix), np.linalg.det(corr_matrix)))
corr_matrix.shape

Ранг матрицы корреляций максимальный, но значение определителя матрицы очень близко к нулю. 

Самая сильная связь цены с мощностью двигателя и объемом двигателя. Также видно, что между мощностью двигателя и объемом двигателя очень высокий коэффициент корреляции  0.87. 

### Оценка значимости числовых переменных.
#### Для оценки значимости числовых переменных будем использовать функцию f_classif из библиотеки sklearn.
Возможности модуля sklearn.feature_selection могут быть использованы не только для выбора важных признаков, но и для уменьшения размерности, улучшения предсказательной силы моделей, либо для повышения их производительности на очень многомерных наборах данных.

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA). Основу процедуры составляет обобщение результатов двух выборочных t-тестов для независимых выборок (2-sample t).

В качестве меры значимости мы будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [ ]:
# Сделаю оценку значимости
imp_num = pd.Series(f_classif(train[train_numeric_cols], train['price'])[0], index = train_numeric_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Из визуализации распределения важности признаков и видно, что мощность двигателя (engine_power) самый значимый показатель по ANOVA F test, потом количество год выпуска модели (model_date) и в конце идентификатор продавца (sell_id)

### Посмотрим на значимость категориальных переменных 
#### Для оценки значимости категориальных переменных будем использовать функцию mutual_info_classif из библиотеки sklearn. Данная функция опирается на непараметрические методы, основанные на оценке энтропии в группах категориальных переменных.

In [ ]:
# Для начала преобразуем данные: 

# Для категориальных признаков применим метод кодирования One-Hot Encoding:
#X_cat = OneHotEncoder(sparse=False).fit_transform(train[train_category_cols].values)

# Для mutual_info_classif не получилось проверить значимость после OneHotEncoder. 
# Прочитала, что можно использовать label_encoder

label_encoder = LabelEncoder()

train_cat = train.copy()

for column in train_category_cols:
    train_cat[column] = label_encoder.fit_transform(train[column])
    
# убедимся в преобразовании    
train_cat.head()

In [ ]:
imp_cat = pd.Series(mutual_info_classif(train_cat[train_category_cols], train['price'],#  
                                     discrete_features=True), index = train_category_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Таким образом, самым значимым признаком по Mutual information тесту является описание (description) и конфигурация транспортного средства (vehicle_configuration), потом идет полное описание (super_gen) и в конце валюта (price_currency).

## Подготовка данных к машинному обучению

### Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [88]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import re
from datetime import datetime

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor
from sklearn.cluster import KMeans
from sklearn.base import clone
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import cross_validate

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")


# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы!
RANDOM_SEED = 42

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

# Объявим функции:
def param_data(data): # посмотрим на данные
  param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              'loc[1]': data.loc[1].values,
              }, 
             index = data.loc[0].index)
  return param


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


# Функция переименования столбцов 

def upper_letter(word):
    for i in range(len(word)):
        if word[i].isupper() and i!=0:
            word = word[:i] + ' ' + word[i:].lower()
    return word.replace(' ', '_').lower()

# Функция от Владимира Юшманова
def get_perf_type(x, body_type_list):
    for t in body_type_list:
        if t in x:
            return t
        else: continue
    else: return '0'

# Функция от Vandr, которая сделает полный список всех возможных характеристик в тестовой выборке

def get_test_features(equipment):
    # Создаем пустой список, в который будут добавляться все характеристики
    all_features = []
    for data in equipment:
        # Находим все слова между кавычками
        features=re.findall(r'\"(.+?)\"',data)
        # Добавляем в общий список
        all_features.extend(features)
    # Удаляем дубликаты
    all_features = list(dict.fromkeys(all_features))
    return all_features


# Функция замены значения параметра equipment в тестовой выборке на список характеристик от Vandr 

def get_features_test(equipment): 
    features=re.findall(r'\"(.+?)\"',equipment)  
    return features


# Функция по созданию нового признака:

def creating_new_columns(df, column, function):
    if function == 'log':
        df[df[column].name+'_log'] = np.log(df[column][df[column] > 0])
        df[df[column].name+'_log'].fillna(0, inplace=True)
    elif function=='sqrt':
        df[df[column].name+'_sqrt'] = np.sqrt(df[column][df[column] > 0])
        df[df[column].name+'_sqrt'].fillna(0, inplace=True)
    elif function =='isNAN':
        if df[column].isnull().any() == True:
            df[df[column].name+'_isNAN'] = pd.isna(df[column]).astype('uint8')
            

# Создадим необходимые словари

# Словарь для color
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}

# Словарь для owners обучающей выборки
dict_train_owners = {'4.0': 4, '3.0': 3, '2.0': 2, '1.0': 1, '0.0': 0}

# Словари для замены русских значений на английские в признаках 
# rudder, pts, fuel_type, vehicle_transmission и drive тестовой выборки
dict_rudder = {'Левый': 'LEFT',
               'Правый': 'RIGHT'}

dict_pts = {'Оригинал': 'ORIGINAL', 
            'Дубликат': 'DUPLICATE'}

dict_test_fuel = {'бензин': 'GASOLINE', 
                  'гибрид': 'HYBRID', 
                  'дизель': 'DIESEL',
                  'электро': 'ELECTRO',
                  'газ': 'LPG'}

dict_transmission = {'автоматическая': 'AUTOMATIC',
                     'механическая': 'MECHANICAL',
                     'роботизированная': 'ROBOT',
                     'вариатор': 'VARIATOR'}

dict_drive = {'передний': 'FORWARD_CONTROL',
              'задний': 'REAR_DRIVE',
              'полный': 'ALL_WHEEL_DRIVE'}

# Словарь owners для тестовой выборки 
dict_owners = {'3 или более': 3, '1\xa0владелец': 1, '2\xa0владельца': 2}

In [89]:
# на всякий случай, заново подгружаем данные
VERSION    = 15
DIR_TRAIN  = '/kaggle/input/parsingautoru24122020/' # подключила к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

!ls '../input'

# Загружаем train
train = pd.read_csv(DIR_TRAIN+'all_auto_ru__24_12_2020.csv', sep=';') # датасет для обучения модели
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

# Загружаем test
test = pd.read_csv(DIR_TEST+'test.csv')

# Переименовываем столбцы тестовой выборки
test.rename(columns=lambda x: upper_letter(x) if re.search('[a-z]', x) else x, inplace=True)

russian_cols = {'Владельцы': 'owners', 'Владение': 'own', 'ПТС': 'pts', 'Привод': 'drive', 
                'Руль': 'rudder', 'Состояние': 'condition', 'Таможня': 'customs'}

test.rename(columns=russian_cols, inplace=True)
#____________________________________
engine_displacement_tmp = test['engine_displacement'].str.split(" ", expand = True)
test['engine_displacement'] = engine_displacement_tmp[0]
test.loc[test['engine_displacement'] == '', 'engine_displacement'] = 0

#____________________________________
summary_tmp = train['summary'].str.replace(',', '').str.split(" ", expand = True)
summary_tmp[0] = summary_tmp[0].str.replace('[A-Z,a-z]', '').str.strip()
summary_tmp = summary_tmp[0]
train = pd.concat([train,summary_tmp], axis=1)
train['engine_displacement'] = train[0]
train = train.loc[train['engine_displacement'] != '']

parsing-all-moscow-auto-ru-09-09-2020  sf-dst-car-price-prediction
parsingautoru24122020


In [90]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка #################################################
    
    # Обработка body_type 
    # возьму данные из test
    test['bodytype'] = [str(x).lower().replace('.', '') for x in test['body_type']]
    # Сделаю список типов кузовов
    body_type_list = list(test['bodytype'].unique())

    # Заполню пропущенное значение в признаке pts самым частовстречающимся значением
    df_output.loc[df_output['pts'].isna(), 'pts'] = df_output['pts'].mode()[0]

    if 'price' in df_output.columns:
        df_output['bodytype'] = [str(x).lower() for x in df_output['body_type']]
        df_output['brand'] = df_output['brand'].apply(lambda x: x.strip().upper())
        df_output['color'] = df_output['color'].map(dict_color)
        df_output.loc[(df_output['fuel_type'] == 'CLA_KLASSE'), 'fuel_type'] = 'GASOLINE'
        df_output['owners'].fillna(0, inplace=True)
        df_output['owners'] = df_output['owners'].astype('string').map(dict_train_owners)
        df_output.description.fillna('Отсутствует', inplace=True)
        df_output.dropna(subset=['price'], inplace=True)
        df_output.drop(['mark', 'model', 'summary',0], axis=1, inplace=True)
    else:
        df_output['bodytype'] = [str(x).lower().replace('.', '') for x in test['body_type']]
        df_output['rudder'] = df_output['rudder'].map(dict_rudder)
        df_output['pts'] = df_output['pts'].map(dict_pts)
        df_output['fuel_type'] = df_output['fuel_type'].map(dict_test_fuel)
        df_output['vehicle_transmission'] = df_output['vehicle_transmission'].map(dict_transmission)
        df_output['drive'] = df_output['drive'].map(dict_drive)
        #engine_displacement_tmp = df_output['engine_displacement'].str.split(" ", expand = True)
        #df_output['engine_displacement'] = engine_displacement_tmp[0]
        #df_output.loc[test['engine_displacement'] == '', 'engine_displacement'] = 0
        #df_output['engine_displacement'] = df_output['engine_displacement'].astype(int)
        engine_power_tmp = df_output['engine_power'].str.split(" ", expand = True)
        df_output['engine_power'] = engine_power_tmp[0]
        df_output['engine_power'] = df_output['engine_power'].astype(int)
        df_output['owners'] = df_output['owners'].map(dict_owners)
        
    # Заменим длинные типы в train'е на обобщенные значения
    df_output['bodytype'] = df_output['bodytype'].apply(lambda x: get_perf_type(x, body_type_list))
        

    # Удалим предложения, для которых не нашлось соответствия типа кузова
    df_output = df_output[df_output['bodytype']!='0']
    df_output['body_type'] = df_output['bodytype']
    
    # Преобразую тип данных к float
    df_output['engine_displacement'] = df_output['engine_displacement'].astype(float)
    
    # преобразую тип данных к int
    df_output['owners'] = df_output['owners'].astype(int)
    df_output['number_ofdoors'] = df_output['number_ofdoors'].astype(int)

    # Удалю "лишние признаки" 
    df_output.drop(['bodytype', 'car_url', 'complectation_dict', 'equipment_dict', 'image', 'condition', 
                    'customs', 'equipment_dict', 'model_info', 'model_name', 'name', 'own', 'parsing_unixtime',
                    'price_currency', 'vehicle_configuration', 'super_gen', 'sell_id'], axis=1, inplace=True) # 


    # Распределю признаки по категориям: категориальные и числовые
    df_output_cols = df_output.columns#[1:-2]
    output_category_cols, output_numeric_cols = [], []

    for j in df_output_cols:
        if j!='price':
            if df_output[j].dtypes!=object:
                output_numeric_cols.append(j)
            else:
                output_category_cols.append(j)
            
    # Для каждого числового признака создам новый признак, содержащий логарифмированную величину этой переменной
    output_numeric_log, func = [], 'log'

    for i in output_numeric_cols:
        creating_new_columns(df_output, i, func)
        output_numeric_log.append(i+'_'+func)
    
    # Для каждого числового признака создам новый признак, содержащий корень величины этой переменной
    output_numeric_sqrt, func = [], 'sqrt'

    for i in output_numeric_cols:
        creating_new_columns(df_output, i, func)
        output_numeric_sqrt.append(i+'_'+func)

    #___________________________________________

    # Добавлю к списку числовых признаков логарифмы и квадратичные признаки
    output_numeric_cols = output_numeric_cols + output_numeric_log + output_numeric_sqrt

            
    print('category_columns =', output_category_cols, '-',len(output_category_cols), 
          '\nnumeric_columns =', output_numeric_cols)
    
    # Будем запускать отдельно для train и отдельно для test
    
    if 'price' in df_output.columns:
        Y = df_output['price'].values
        df_output.drop(['price'], axis=1, inplace=True)
        return df_output, Y, output_category_cols, output_numeric_cols
    else:
        return df_output, output_category_cols, output_numeric_cols

# Построение модели

In [91]:
# Запускаем отдельно для train и для test. Проверяем, что получилось:
train_df, y, category_cols, numeric_cols = preproc_data(train)
test_df, category_cols, numeric_cols = preproc_data(test)
print(train_df.shape, test_df.shape)

category_columns = ['body_type', 'brand', 'color', 'description', 'fuel_type', 'vehicle_transmission', 'vendor', 'pts', 'drive', 'rudder'] - 10 
numeric_columns = ['engine_displacement', 'engine_power', 'mileage', 'model_date', 'number_ofdoors', 'production_date', 'owners', 'engine_displacement_log', 'engine_power_log', 'mileage_log', 'model_date_log', 'number_ofdoors_log', 'production_date_log', 'owners_log', 'engine_displacement_sqrt', 'engine_power_sqrt', 'mileage_sqrt', 'model_date_sqrt', 'number_ofdoors_sqrt', 'production_date_sqrt', 'owners_sqrt']
category_columns = ['body_type', 'brand', 'color', 'description', 'fuel_type', 'vehicle_transmission', 'vendor', 'pts', 'drive', 'rudder'] - 10 
numeric_columns = ['engine_displacement', 'engine_power', 'mileage', 'model_date', 'number_ofdoors', 'production_date', 'owners', 'engine_displacement_log', 'engine_power_log', 'mileage_log', 'model_date_log', 'number_ofdoors_log', 'production_date_log', 'owners_log', 'engine_displacement_sqrt'

In [ ]:
train_df.head(1)

In [ ]:
test_df.head(2)

In [ ]:
def encoding_data(df_input, cat_cols, num_cols):
    df_output = df_input.copy()
    # Преобразуем данные: 
    label_encoder = LabelEncoder()

    X_cat = pd.DataFrame()

    for column in cat_cols:
        X_cat[column] = label_encoder.fit_transform(df_output[column])#X_cat[column].astype('category').cat.codes
        
    # Для категориальных признаков применим метод кодирования One-Hot Encoding:
    #X_cat = OneHotEncoder(sparse = False).fit_transform(df_output[cat_cols].values)

    # Стандартизируем числовые переменные:
    X_num = StandardScaler().fit_transform(df_output[num_cols].values)

    # Объединим стандартизованные числовые и закодированные категориальные переменные 
    # в одно признаковое пространство
    df_output = np.hstack([X_num, X_cat])
    return df_output

    #return df_output

In [ ]:
train_data = encoding_data(train_df, category_cols, numeric_cols)
test_data = encoding_data(test_df, category_cols, numeric_cols)
print(train_data.shape, test_data.shape)

In [ ]:
train_data

# Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по модели двигателя (engineDisplacement). 
C ней будем сравнивать другие модели.




## Train Split

In [ ]:
# для baseline просто возьму пару схожих признаков без полной обработки
columns = ['body_type', 'brand', 'production_date', 'engine_displacement', 'mileage']
df_train = train_df[columns]
df_test = test_df[columns]


# Объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


for colum in ['body_type', 'brand', 'engine_displacement']:
    data[colum] = data[colum].astype('category').cat.codes

X = data.query('sample == 1').drop(['sample'], axis=1)
X_sub = data.query('sample == 0').drop(['sample'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [ ]:
X_train

In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
tmp_train.groupby('engine_displacement')['price'].median()

In [ ]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engine_displacement'].map(tmp_train.groupby('engine_displacement')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# # Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Простая модель с тремя признаками

In [ ]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.save_model('catboost_single_model_like_baseline.model')

In [ ]:
predict = model.predict(X_test)

# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 9.18%!

## Построение модели на всех признаках
---
### Разбиваем датасет на тренировочный и тестовый

In [ ]:
X = train_df
X_sub = test_df

# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# проверяем
test_df.shape, train_df.shape, X.shape, X_train.shape, X_test.shape


## Обучаем модель, генерируем результат и сравниваем с тестом
### Первая модель

In [ ]:
model = CatBoostRegressor(iterations = 50,
                          cat_features=category_cols,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         use_best_model=True,
         verbose=False,
         plot=True
         )

model.save_model('catboost_model.model')

predict = model.predict(X_test)
# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

## Submission

Курс доллара изменился с сентября (данные тестовой выборки) по декабрь (данные обучающей выборки) на 3.25%. 
Поэтому в сабмит сделаю поправку на изменение курса доллара.

In [ ]:
predict_submission_cat = np.round(model.predict(X_sub)*0.9685)
sample_submission['price'] = predict_submission_cat.astype(int)
sample_submission.to_csv('submission_model_1.csv', index=False)
sample_submission.head(10)

С этой моделью на лидеборде score стало 42.94

## Вторая модель. Подбор параметров.

Попробую логарифмировать price

In [92]:
# Подготовим данные. 

copy_train = train_df.copy()
copy_test = test_df.copy()

# Объединяем трейн и тест в один датасет
copy_train['sample'] = 1 # помечаем где у нас трейн
copy_test['sample'] = 0 # помечаем где у нас тест

copy_data = copy_test.append(copy_train, sort=False).reset_index(drop=True) # объединяем

for column in category_cols:
    copy_data[column] = copy_data[column].astype('category').cat.codes
    
X = copy_data.query('sample == 1').drop(['sample'], axis=1)
X_sub = copy_data.query('sample == 0').drop(['sample'], axis=1)

# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# проверяем
test_df.shape, X_sub.shape, X.shape, X_train.shape, X_test.shape


((34686, 31), (34686, 31), (126205, 31), (100964, 31), (25241, 31))

In [ ]:
model = CatBoostRegressor(iterations = 50,
                          cat_features=category_cols,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
grid = {'learning_rate': [ 0.13, 0.14, 0.15]
        ,'depth': [12]
        ,'l2_leaf_reg': [7, 7.5, 8]
        ,'random_strength': [0.3]}

model_cb = CatBoostRegressor(iterations = 5000,
                       random_seed = RANDOM_SEED,
                       eval_metric='MAPE',
                       custom_metric=['R2', 'MAE'],
                       silent=True,
                       learning_rate=0.13, depth=12,
                       l2_leaf_reg=8, random_strength=0.3)

model_cb.fit(X_train, np.log(y_train),
         eval_set=(X_test, np.log(y_test)),
         verbose=False,
         use_best_model=True,
         plot=True)

model_cb.save_model('catboost_log_model.model')

predict_exp = np.round(np.exp(model_cb.predict(X_test)))
# оцениваем точность
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_exp))*100:0.2f}%")

## Submission

In [ ]:
predict_log_submission = np.round(np.exp(model_cb.predict(X_sub))*0.9685)
sample_submission['price'] = predict_log_submission.astype(int)
sample_submission.to_csv('submission_model_2.csv', index=False)
sample_submission.head(10)

На лидеборде score стало 28.16.

После корректировки цены с поправкой на изменение курса доллара со времени тестовой выборки, score на ЛБ стало 27.94
После удаления признаков price_currency и parsing_unixtime score стало 22.03

## Кросс-валидация (CV)

Когда мы делаем отбор признаков или перебираем настройки модели, мы постоянно смотрим в тестовые данные (X_test), что может привести к подгону под тестовые данные. В итоге мы получим Переобучение (overfitting).
Чтобы избежать этого, можно сразу использовать кросс-валидацию по фолдам.

Ниже представлен Пример, как можно организовать обучение модели на 5 фолдах, с дальнейшим объединением предсказаний от каждой модели.

In [ ]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

idx = np.argsort(model_cb.feature_importances_)

In [ ]:
cat_features_ids = np.where(X_train.dtypes == object)[0].tolist()

submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

### Submissions blend

In [ ]:
submissions

In [ ]:
submissions['blend'] = (submissions.sum(axis=1)/len(submissions.columns))*0.9685
sample_submission['price'] = np.round(submissions['blend'].values)
sample_submission.to_csv('submission_blend_2.csv', index=False)
sample_submission.head(10)

Результат ухудшился, score на лидеборде стал 42.54

## Random Forrest

In [ ]:

random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rfr = RandomForestRegressor(random_state = RANDOM_SEED)

rf_random = RandomizedSearchCV(estimator = rfr, 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=10, 
                               random_state=RANDOM_SEED, 
                               n_jobs = -1)
rf_random.fit(X_train, np.log(y_train))
print(rf_random.best_params_)

#best_params_: 
#{'n_estimators': 300,
# 'min_samples_split': 10,
# 'min_samples_leaf': 1,
# 'max_features': 'sqrt',
# 'max_depth': None,
# 'bootstrap': False}



In [ ]:
#best_rfr = rf_random.best_estimator_
best_rfr = RandomForestRegressor(random_state=RANDOM_SEED
                      , n_estimators=300
                      , min_samples_split=10
                      , min_samples_leaf=1
                      , max_features='sqrt'
                      , max_depth=None
                      , bootstrap=False)

best_rfr.fit(X_train, np.log(y_train))


predict_rfr = np.exp(best_rfr.predict(X_test))
print(y_test, predict_rfr)

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_rfr))*100:0.2f}%")
predict_rfr_submission = np.round(np.exp(best_rfr.predict(X_sub))*0.9685)
sample_submission['price'] = predict_rfr_submission.astype(int)
sample_submission.to_csv('submission_model_rfr.csv', index=False)
sample_submission.head(10)

В результате score на ЛБ 19.05

# Stacking
## Как каждая модель относится к категориальным переменным?
CatBoost обладает гибкостью, позволяя задавать индексы категориальных столбцов, чтобы его можно было кодировать как кодирование в одно касание с использованием one_hot_max_size (используйте кодирование в одно касание для всех функций с числом различных значений, меньшим или равным данному значению параметра).

Если вы ничего не передаете в аргументе cat_features, CatBoost будет обрабатывать все столбцы как числовые переменные.

Примечание. Если в cat_features не указан столбец со строковыми значениями, CatBoost выдает ошибку. Кроме того, столбец с типом int по умолчанию будет считаться числовым по умолчанию, его необходимо указать в cat_features, чтобы алгоритм воспринимал его как категориальный.


Как и в CatBoost, LightGBM также может обрабатывать категориальные функции, вводя имена функций. Он не конвертируется в одноразовое кодирование и намного быстрее, чем одноразовое кодирование. LGBM использует специальный алгоритм, чтобы найти значение разделения категориальных признаков.

Примечание. Перед построением набора данных для LGBM вы должны преобразовать свои категориальные функции в тип int. Он не принимает строковые значения, даже если вы передаете его через параметр categoryorical_feature.


В отличие от CatBoost или LGBM, XGBoost не может обрабатывать категориальные функции сам по себе, он принимает только числовые значения, подобные случайному лесу. Поэтому перед подачей категориальных данных в XGBoost необходимо выполнить различные кодировки, такие как кодирование меток, среднее кодирование или однократное кодирование.


RandomForestRegressor, LinearRegression - им тоже нужны на вход данные без категориальных признаков

## Подготовка


In [ ]:
# Снова объединю копии обучающей и тестовой выборок
copy_data = copy_test.append(copy_train, sort=False).reset_index(drop=True) # объединяем

copy_data.head()


In [ ]:
# Сделаю теперь dummies-преобразование 
for column in category_cols:
    dummies = pd.get_dummies(copy_data[column], prefix = copy_data[column].name)

    # Удаляем исходный столбец и добавляем dummies
    copy_data = copy_data.drop(copy_data[column].name, axis=1).join(dummies)

copy_data.head(3)

Dummies-преобразование сделать не получилось (выходит ошибка переполнения памяти). Поэтому оставлю датасеты как есть.

In [ ]:
for column in category_cols:
    copy_data[column] = copy_data[column].astype('category').cat.codes

In [ ]:
# Теперь разделяем обучающую и тестовую выборки

X = copy_data.query('sample == 1').drop(['sample'], axis=1)
X_sub = copy_data.query('sample == 0').drop(['sample'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Model 1

In [ ]:
def compute_meta_feature(model, X_train, X_test, y_train, cv):
   
    X_meta_train = np.zeros_like(y_train, dtype = np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_model = clone(model)
        folded_model.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)
        
    meta_model = clone(model)
    meta_model.fit(X_train, y_train)
    
    X_meta_test = meta_model.predict_proba(X_test)[:,1]
    
    return X_meta_train, X_meta_test

In [ ]:
cv = KFold(n_splits=N_FOLDS, shuffle=True)

In [ ]:
# 1 - Catboost

cat_features_ids = np.where(X.dtypes == object)[0].tolist()

X_meta_train_features = []
X_meta_test_features = []

model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          loss_function = 'RMSE'
                         )

X_meta_train = np.zeros_like(y, dtype = np.float32)
X_meta_test = np.zeros(len(X_sub), dtype = np.float32)
for train_fold_index, predict_fold_index in cv.split(X):
    X_fold_train, X_fold_predict = X.iloc[train_fold_index], X.iloc[predict_fold_index]
    y_fold_train = y[train_fold_index]

    folded_model = clone(model)
    folded_model.fit(X_fold_train, y_fold_train,
                     cat_features=cat_features_ids,
                     eval_set=(X_test, y_test),
                     verbose_eval=1000,
                     use_best_model=True,
                     plot=False
)
    X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)
    X_meta_test += folded_model.predict(X_sub)

X_meta_test = X_meta_test / N_FOLDS

X_meta_train_features.append(X_meta_train)
X_meta_test_features.append(X_meta_test)

print(model.get_best_score())

In [ ]:
# 2 - RandomForestRegressor

model = RandomForestRegressor(n_estimators=400, 
                              random_state=42, 
                              min_samples_split=10, 
                              min_samples_leaf=1, 
                              max_features='sqrt')

X_meta_train = np.zeros_like(y, dtype = np.float32)
X_train_num = X
X_sub_num = X_sub

for train_fold_index, predict_fold_index in cv.split(X_train_num):
    X_fold_train, X_fold_predict = X_train_num.iloc[train_fold_index], X_train_num.iloc[predict_fold_index]
    y_fold_train = y[train_fold_index]

    folded_model = clone(model)
    folded_model.fit(X_fold_train, y_fold_train)
    X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)

meta_model = clone(model)
meta_model.fit(X_train_num, y)

X_meta_test = meta_model.predict(X_sub_num)

X_meta_train_features.append(X_meta_train)
X_meta_test_features.append(X_meta_test)

In [ ]:
X_meta_test_features

In [ ]:
# 3 LinearRegression

model = LinearRegression(normalize = True)

X_meta_train = np.zeros_like(y, dtype = np.float32)

for train_fold_index, predict_fold_index in cv.split(X_train_num):
    X_fold_train, X_fold_predict = X_train_num.iloc[train_fold_index], X_train_num.iloc[predict_fold_index]
    y_fold_train = y[train_fold_index]

    folded_model = clone(model)
    folded_model.fit(X_fold_train, y_fold_train)
    X_meta_train[predict_fold_index] = folded_model.predict(X_fold_predict)

meta_model = clone(model)
meta_model.fit(X_train_num, y)

X_meta_test = meta_model.predict(X_sub_num)

X_meta_train_features.append(X_meta_train)
X_meta_test_features.append(X_meta_test)

In [ ]:
stacked_features_train = np.vstack(X_meta_train_features[:2]).T
stacked_features_test = np.vstack(X_meta_test_features[:2]).T

### В качестве финальной модели используем линейную регрессию.

In [ ]:
final_model = LinearRegression()
final_model.fit(stacked_features_train, y)

## Submission

In [ ]:
sample_submission['price'] = np.floor(final_model.predict(stacked_features_test)*0.9685 / 10000) * 10000 
sample_submission.to_csv('submission_stack_model_1.csv', index=False)
sample_submission.head(10)

## Model 2

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict
from sklearn.model_selection import KFold


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features usinf the classifier cls
    
    :arg model: scikit-learn classifier
    :arg X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    
    X_meta_train = np.zeros_like(y_train, dtype = np.float32)
    X_meta_test = np.zeros(len(X_test), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
            
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
        
        print_regression_metrics(X_meta_train[predict_fold_index], y_train[predict_fold_index])
        X_meta_test += folded_clf.predict(X_test)
    
    n = cv.n_splits
    X_meta_test = X_meta_test / n
    
    return X_meta_train, X_meta_test

In [ ]:
def generate_meta_features(regressors, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(regressors)
    ]
    
    stacked_features_train = np.stack([
        features_train for features_train, features_test in features
    ], axis=-1)

    stacked_features_test = np.stack([
        features_test for features_train, features_test in features
    ], axis=-1)
    
    return stacked_features_train, stacked_features_test

In [ ]:
cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return print_regression_metrics(y_test, y_test_pred)

In [ ]:
from sklearn.base import clone

from sklearn.preprocessing import StandardScaler
# Стандартизируем данные:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
test = scaler.fit_transform(X_sub)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED),
    BaggingRegressor(ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_SEED)),
    CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.005,
                         iterations=4500,
                         l2_leaf_reg=2,
                         depth=6,
                         bootstrap_type = 'Bayesian',
                         random_seed=42,
                         od_type='Iter',
                         od_wait=100)
    ], X_train, test, y_train, cv)


#Строим мета-алгоритм

final_model = LinearRegression()
final_model.fit(stacked_features_train, y_train)

## Submission

In [ ]:
y_pred = np.round((final_model.predict(stacked_features_test)/1000))*1000

sample_submission['price'] =  np.round(y_pred*0.9685)
sample_submission.to_csv('submission_stack_model_2.csv', index=False)

sample_submission.head(10)

## Model 3

In [ ]:
# Подготовим данные. 

# Снова объединяем копии трейн и тест в один датасет
copy_data = copy_test.append(copy_train, sort=False).reset_index(drop=True) # объединяем

for column in category_cols:
    copy_data[column] = copy_data[column].astype('category').cat.codes
    
X = copy_data.query('sample == 1').drop(['sample'], axis=1)
X_sub = copy_data.query('sample == 0').drop(['sample'], axis=1)

# Прологарифмируем целевой признак
y = np.log(y)

# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Запоминаем порядок колонок
column_list = test_df.columns

# Устанавливаем порядок колонок как для тестовой выборки, иначе предсказания неверные.
X_train = X_train[column_list]
X_test = X_test[column_list]
X_sub =X_sub[column_list]
#y = np.log(train_data.price.values)

# проверяем
test_df.shape, X_sub.shape, X.shape, X_train.shape, y_train.shape[0], X_test.shape, y_test.shape[0]



In [ ]:
import datetime as dt
from vecstack import stacking

from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import ExtraTreesRegressor    
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


# Configure models
RANDOM_SEED = 42


lr = LinearRegression(normalize=True, n_jobs=-1)

etc = ExtraTreesRegressor(n_estimators=500,  n_jobs=-1,
                          random_state=RANDOM_SEED)  # max_depth=5,
catb = CatBoostRegressor(iterations=3500,
                                 learning_rate=0.05,
                                 random_seed=RANDOM_SEED,
                                 eval_metric='MAPE',
                                 verbose = 500
                                 )
rf = RandomForestRegressor(random_state=RANDOM_SEED, n_jobs=-1,
                           n_estimators=500)  # , max_depth=3

knn = KNeighborsRegressor(n_neighbors=5, 
                          weights='uniform', 
                          algorithm='auto', 
                          leaf_size=30, 
                          p=2, metric='minkowski', n_jobs=-1)


print("Finished setting up regressors at ", dt.datetime.now())

# Initialize 1-st level models.
models = [catb, rf, etc, knn]

# Compute stacking features
S_train, S_test = stacking(models, X_train, y_train, X_test,
                           regression=True, metric=mape, n_folds=4,
                           shuffle=True, random_state=RANDOM_SEED, verbose=2)

# Initialize 2-nd level model
model = lr

# Fit 2-nd level model
model = model.fit(S_train, y_train)

# Predict
y_test_pred = np.exp(model.predict(X_test))

## Submission

In [ ]:
sample_submission['price'] = np.round(np.exp(model.predict(X_sub)))
# sample_submission['price'] = sample_submission['price'].apply(lambda x: round(x/1000)*1000)
sample_submission.to_csv('submission_stack_model_3.csv', index=False)
sample_submission.head(10)

## Результат

В итоге получили **MAPE 30%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

## GradientBoostingRegressor

In [93]:
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=1000)
gb.fit(X_train, np.log(y_train))

predict_gb_test = np.exp(gb.predict(X_test))
predict_gb_submission = np.exp(gb.predict(X_sub))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_gb_test))*100:0.2f}%")

Точность модели по метрике MAPE test: 5.12%
Точность модели по метрике MAPE submission: 5.12%


## Submission

In [96]:
sample_submission['price'] = np.round(np.exp(gb.predict(X_sub))*0.9685) #predict_gb_submission
sample_submission.to_csv('submission_gb_model.csv', index=False)
sample_submission.head(10)

,sell_id,price
0,1100575026,640054.0
1,1100549428,807559.0
2,1100658222,839820.0
3,1100937408,770743.0
4,1101037972,845295.0
5,1100912634,744943.0
6,1101228730,627963.0
7,1100165896,422903.0
8,1100768262,1863357.0
9,1101218501,811704.0


# Результат

#### На ЛБ результат стал 17.94

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing